In [1]:
import pyGM as gm
import numpy as np
import pyGM.messagepass

# Basic Error Probabilities

In the first section we will be discussing and analyzing repetition and Hamming error codes. For more information, check here: en.wikipedia.org/wiki/Hamming(7,4)

## Compute the probability that, with no encoding at all, we will make at least one mistake in sending a block of four bits.

The probability of having an error on one bit is p = 0.1

To find the probability that we will make AT LEAST one mistake for one bit, we can calculate the probability:

    pb(errors >= 1) = 1 - p(no errors)

Because the bit reading probability is independent and uniform, we can take the probability of a bit being read correctly and raise it to the power of the number of bits: 

    p(errors >= 1) = 1 - p(no errors)^(num bits) = 1 - 0.9^4 = 0.3439

### The probability that at least one bit is incorrectly read in each 4 bit block is 0.3439


## Now compute the probability that, with a rate 1/3 repetition code, we will make at least one error in the block of four bits.

With a repetition rate 1/3 repetition code a bit is encoded from 0 --> 000, 01 --> 010101

To find the probability that we will make AT LEAST one mistake, we can calculate 1 - p(no error) + p(1st wrong) + p(2nd wrong) + p(3rd wrong)

For any one bit here are the probabilities:

    pb(no errors) = 0.9^3 = 0.729 
    pb(1st wrong) = pb(2nd wrong) = pb(3rd wrong) = 0.1 * 0.9^2 * 3 = 0.243
    
The probability of having one wrong bit is the same regardless of the bit position.

We can use the above values to calculate the probability of no errors for one repetition of each bit:

    p(no errors) = (pb(no errors)+pb(1st wrong))^4 = 0.972
    
With this final value we can calculate the probability of at least one error in a four bit encoded block:

    p(errors >= 1) = 1 - p(no errors)^4 = 1 - 0.972^4 = 0.1074

### The probability that at least one bit is incorrectly read in each 4 bit encoded block is 0.1074


## Now compute the probability that, with a Hamming(7,4) code, we will make at least one error in the four bits. 

With a Hamming(7,4) code a bit is encoded from 0000 --> 0000000 , 0101 --> 0100101

To find the probability that we will make AT LEAST one mistake for one bit, we can calculate 1 - p(no errors) + p(1st wrong) + ... + p(7th wrong)

    p(no errors) = 0.9^7 = 0.4783
    pb(1st wrong) = p(2nd wrong) = ... = p(7th wrong) = 0.1 * 0.9^6 = 0.0531
    
From this we can calculate the probability that there is no errors in the whole block:
    
    p(no errors) = p(1st wrong)*7 + p(no errors) = 0.0531*7 + 0.4783 = 0.85
    
With this final value we can calculate the probability of at least one error in a 4 bit encoded block:

    p(errors >= 1) = 1 - p(no errors) = 1 - 0.85 = 0.15
    
### The probability that at least one bit is incorrectly read in each 4 bit encoded block is 0.15


# Low-Density Parity Check Codes

For more information on LDPCs check out this article: en.wikipedia.org/wiki/Low-density_parity-check_code

Here we will be creating and evaluating a rate 1/2 LDPC code where B = 1000 data bits.

In [2]:
p = 0.1
B = 1000
c = [None]*B
for i in range(B):
    c[i] = np.random.permutation(B)[0:3]
    if not (c[i]==i).any: c[i][0] = i
        
D = np.random.randint(2, size=B)

T = np.copy(D)
for i in range(B):
    T = np.append(T, D[c[i][0]]^D[c[i][1]]^D[c[i][2]])
    
R = np.copy(T)
for i in range(len(T)): R[i] = T[i] if np.random.rand() > p else 1-T[i]

If we had done no encoding at all, we could check how many data bits are incorrect, and also how many sequential blocks of four bits are incorrect.

In [3]:
print('p ~ {}'.format(sum([R[i]!=D[i] for i in range(B)]) /float(B)))

err = 0
for i in range(0,B,4):
    for j in range(4):
        if (R[i+j]!=D[i+j]):
            err += 1
            break
            
print('p(errors >= 1) ~ {}'.format(err/float(B/4)))

p ~ 0.093
p(errors >= 1) ~ 0.332


In [4]:
X = [gm.Var(i,2) for i in range(len(T))]

parity_check = np.zeros((2, 2, 2, 2))
parity_check += 1e-300
parity_check[0, 0, 0, 0] = 1.0
parity_check[0, 0, 1, 1] = 1.0
parity_check[0, 1, 0, 1] = 1.0
parity_check[1, 0, 0, 1] = 1.0
parity_check[0, 1, 1, 0] = 1.0
parity_check[1, 0, 1, 0] = 1.0
parity_check[1, 1, 0, 0] = 1.0
parity_check[1, 1, 1, 1] = 1.0

checks = []
for i,_c in enumerate(c):
    checks.append( gm.Factor([X[_c[0]], X[_c[1]], X[_c[2]], X[B+i]], parity_check))
    
measurements = []
for i in range(len(T)):
    q = p if R[i]==0 else 1.0-p
    measurements.append(gm.Factor([X[i]], [1-q,q]))
    
factors = checks+measurements
fg = gm.GraphModel(factors)

We have now created a graphical model representing the data and parity bits. We will run loopy belief propagation on our model in order to decode it.

In [5]:
lnZ, beliefs = gm.messagepass.LBP(fg, maxIter=15, verbose=True)

Iter 1: 779.1096481139598
Iter 2: -88.35795164355919
Iter 3: -337.9566443822736
Iter 4: -437.5519730373305
Iter 5: -508.0751703733452
Iter 6: -523.9818818816718
Iter 7: -534.4343060319425
Iter 8: -540.9572430247435
Iter 9: -543.0047750368849
Iter 10: -542.8153383606332
Iter 11: -543.2678617627577
Iter 12: -543.3037154377283
Iter 13: -543.2518971903373
Iter 14: -543.195137513773
Iter 15: -543.2063520130553


Finally, we can compute both the "bit error rate" and the "word error rate" for blocks of length 4. 

In [6]:
bit_errs = 0
for t, b in zip(T, beliefs):
    if not t == b.argmax():
        bit_errs += 1
        
word_errs = 0
for i in range(0,B,4):
    for j in range(4):
        if (beliefs[i+j].table.argmax()!=D[i+j]):
            word_errs += 1
            break
        
print('Bit error rate: {}'.format(bit_errs/len(T)))
print('Word error rate: {}'.format(word_errs/float(B/4)))

Bit error rate: 0.042
Word error rate: 0.164
